In [10]:
#Importação do Kaggle
import kagglehub

path = kagglehub.dataset_download("imbikramsaha/dog-breeds")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dog-breeds


In [11]:
!pip install split-folders

In [12]:
#Importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
import cv2, pathlib, splitfolders, os, math
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [13]:
#Arquivos no diretório
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_141.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_149.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_113.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_35.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_152.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_174.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_19.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_25.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_110.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_17.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_67.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_118.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_58.jpg
/kaggle/input/dog-breeds/dog-breeds/basset_hound/basset_hound_84.jpg
/kaggle/input/dog-breeds/do

In [15]:
#Carragamento
DATASET_PATH = '/kaggle/input/dog-breeds/dog-breeds'
IMG_SIZE = 224
cnn_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [ ]:
#Extraindo features das imagens
def extract_features_from_folder(folder_path):
    X, y = [], []
    for breed in os.listdir(folder_path):
        breed_path = os.path.join(folder_path, breed)
        if not os.path.isdir(breed_path):
            continue
        for img_file in tqdm(os.listdir(breed_path), desc=breed):
            img_path = os.path.join(breed_path, img_file)
            try:
                img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = preprocess_input(img_array)
                features = cnn_model.predict(img_array, verbose=0)
                X.append(features.flatten())
                y.append(breed)
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
                continue
    return np.array(X), np.array(y)


print("Extraindo features das imagens...")
X, y = extract_features_from_folder(DATASET_PATH)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Extraindo features das imagens...


english_cocker_spaniel:  27%|██▋       | 54/200 [00:19<00:51,  2.86it/s]

In [ ]:
#Normalização, necessária para o SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Codificação de variável categórica
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print("Original y_train:", y_train[:5])
print("Encoded y_train:", y_train_encoded[:5])
print("Original y_test:", y_test[:5])
print("Encoded y_test:", y_test_encoded[:5])

In [ ]:
#Números referentes às raças
print("\nMapping of encoded numbers to breed names:")
for i, breed in enumerate(label_encoder.classes_):
    print(f"{i}: {breed}")

In [ ]:
# Estatísticas descritivas
breed_counts = pd.Series(y).value_counts()
print("\nEstatísticas descritivas da contagem de imagens por raça:")
print(breed_counts.describe())

# Gráfico de colunas número de imagens por raça
plt.figure(figsize=(15, 10))
breed_counts.plot(kind='bar', color='skyblue')

plt.title('Images per breed')
plt.xlabel('Dog breeds')
plt.xticks(rotation=45)
plt.ylabel('Images')
plt.tight_layout()
plt.show()

# Insights Iniciais
print(f"- O dataset possui um total de {len(y)} imagens.")
print(f"- Existem {len(breed_counts)} raças de cachorro únicas no dataset.")
print(f"- A raça com mais imagens é '{breed_counts.index[0]}' com {breed_counts.iloc[0]} imagens.")
print(f"- A raça com menos imagens é '{breed_counts.index[-1]}' com {breed_counts.iloc[-1]} imagens.")
print(f"- As features extraídas têm {X.shape[1]} dimensões por imagem.")

try:
    print(f"- O dataset foi dividido em treino ({X_train.shape[0]} imagens) e teste ({X_test.shape[0]} imagens) com uma proporção de 80%/20%.")
    print("\nAnálise Exploratória dos Dados (Treino Features Escaladas)")
    print(f"Valor mínimo: {np.min(X_train_scaled):.4f}")
    print(f"Valor máximo: {np.max(X_train_scaled):.4f}")
    print(f"Valor médio: {np.mean(X_train_scaled):.4f}")
    print(f"Desvio padrão: {np.std(X_train_scaled):.4f}")

except NameError:
    print("\nTrain/test split variables (X_train, X_test, X_train_scaled, X_test_scaled) not found. Please run the relevant cells.")

In [ ]:
# Estatísticas descritivas das features
print(pd.DataFrame(X_train_scaled).describe())

print("\nAnálise de Outliers:")
print("- Identificar e analiá-los em CNNs torna-se algo complexo.")
print("- Os valores extremos podem representar características inerentes às imagens.")
print("- A escalabilidade  já atenua a influência de outliers nos algoritmos sensíveis.")

print("\nVerificando Colunas Irrelevantes (Variância zero nas features escaladas):")
variances = np.var(X_train_scaled, axis=0)
zero_variance_features = np.where(variances < 1e-9)[0]

if len(zero_variance_features) > 0:
    print(f" Foram encontradas {len(zero_variance_features)} features com variância próxima de zero:")
    print(zero_variance_features)
    print("Essas features podem ser consideradas irrelevantes, pois não variam entre as amostras de treino.")

else:
    print(" Sem features com variância zero.")

# **Random Forest Classifier**

In [ ]:
#Teste de 20%, recomendado para o dataset.
train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Definindo Pipeline com oversampling e classifier RandomForest
pipeline_rf = Pipeline([
    ('oversample', RandomOverSampler(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Ajustando o grid de hiperparâmetros
param_grid_rf = {
    'classifier__n_estimators': [200],  # Número de árvores na floresta
    'classifier__max_depth': [None, 10, 20],      # Profundidade máxima das árvores
    'classifier__min_samples_split': [2, 5, 10],  # Número mínimo de samples para divisão do nó
    'classifier__min_samples_leaf': [1, 2, 4],   # Número mínimo de samples para formar folha
    'classifier__bootstrap': [True]       # Samples bootstrap para construir árvores
}

# Configurando o GridSearchCV com validação cruzada
grid_search_rf = GridSearchCV(
    pipeline_rf,
    param_grid_rf,
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',  # Métrica de avaliação
    n_jobs=-1,  # Usar cores
    verbose=2  # Exibir progresso
)

print(" Realizando ajuste de hiperparâmetros para Random Forest com validação cruzada...")

# Executando o GridSearchCV nos dados de treino escalados e codificados
grid_search_rf.fit(X_train_scaled, y_train_encoded)

print("\n Ajuste de hiperparâmetros concluído!")

# Melhores parâmetros
print(grid_search_rf.best_params_)

# Melhor acurácia
print(f"{grid_search_rf.best_score_:.4f}")

In [ ]:
#Treino do modelo RF
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42, bootstrap=True, min_samples_leaf=1, min_samples_split=2,verbose=2)
print(" Treinando Random Forest Classifier")
rf_classifier.fit(X_train_scaled, y_train_encoded)

print("Random Forest Classifier training complete.")

In [ ]:
#Previsões
y_pred = rf_classifier.predict(X_test_scaled)

In [ ]:
#Plotando matriz de confusão RF
matriz_confusao = confusion_matrix(y_test_encoded, y_pred)
plt.figure(figsize=(20, 18))
sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Greens', cbar=True, annot_kws={"size": 16})
plt.xlabel('Valores previstos RFC', fontsize=25)
plt.ylabel('Valores reais RFC', fontsize=25)
plt.show()

In [ ]:
#Relatório
report_rfc = classification_report(y_test_encoded, y_pred, output_dict=True)
df_report_rfc = pd.DataFrame(report_rfc).transpose()
print("\nClassification Report (RFC) as DataFrame:")
df_report_rfc

In [ ]:
#Calculando RMSE
rmse_rfc = np.sqrt(mean_squared_error(y_test_encoded, y_pred))
print(f"\nRMSE (RFC): {rmse_rfc:.4f}")

In [ ]:
#Calculando MAE
mae_rfc = mean_absolute_error(y_test_encoded, y_pred)
print(f"\nMAE (RFC): {mae_rfc:.4f}")

# **SVM Classifier**

In [ ]:
#Teste de 20%, recomendado para o dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Defininindo Pipeline com oversampling e classifier SVM
pipeline_svm = Pipeline([
    ('oversample', RandomOverSampler(random_state=42)),
    ('classifier', SVC(random_state=42))
])

# Ajustando o grid de hiperparâmetros
param_grid_svm = {
    'classifier__C': [0.1, 1, 10],  # Parâmetro de regularização
    'classifier__kernel': ['linear', 'rbf'], # Tipos de kernel
    'classifier__gamma': ['scale', 'auto']  # Coeficientes 'rbf', 'poly' e 'sigmoid'
}

#Configurando o GridSearchCV com validação cruzada
grid_search_svm = GridSearchCV(
    pipeline_svm,
    param_grid_svm,
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',  # Métrica de avaliação
    n_jobs=-1,  # Usar todos os cores disponíveis
    verbose=2  # Exibir progresso
)

print("\n Realizando ajuste de hiperparâmetros para SVM com validação cruzada...")

# Executando o GridSearchCV nos dados de treino escalados e codificados
grid_search_svm.fit(X_train_scaled, y_train_encoded)

print("\n Ajuste de hiperparâmetros concluído!")

#Melhores parâmetros
print(grid_search_svm.best_params_)

#Melhor acurácia
print(f"{grid_search_svm.best_score_:.4f}")


 Realizando ajuste de hiperparâmetros para SVM com validação cruzada...
Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
#Treino do modelo SVM
svm_classifier = SVC(kernel='linear', random_state=42, verbose=2,gamma='scale', C=0.1)
print(" Treinando SVM Classifier")
svm_classifier.fit(X_train_scaled, y_train_encoded)
print("SVM Classifier training complete.")

In [ ]:
#Previsões
y_pred_svm = svm_classifier.predict(X_test_scaled)

In [ ]:
#Plotando matriz de confusão SVM
matriz_confusao = confusion_matrix(y_test_encoded, y_pred_svm)
plt.figure(figsize=(20, 18))
sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues', cbar=True, annot_kws={"size": 16})
plt.xlabel('Valores previstos SVM', fontsize=25)
plt.ylabel('Valores reais SVM', fontsize=25)
plt.show()

In [ ]:
#Relatório
report_svm = classification_report(y_test_encoded, y_pred_svm, output_dict=True)
df_report_svm = pd.DataFrame(report_svm).transpose()
print("\nClassification Report (SVM) as DataFrame:")
df_report_svm

In [ ]:
#Calculando RMSE
rmse_svm = np.sqrt(mean_squared_error(y_test_encoded, y_pred_svm))
print(f"\nRMSE (SVM): {rmse_svm:.4f}")

In [ ]:
#Calculando MAE
mae_svm = mean_absolute_error(y_test_encoded, y_pred_svm)
print(f"\nMAE (SVM): {mae_svm:.4f}")

# **XGboost**

In [ ]:
#Teste de 20%, recomendado para o dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("\nTreinando XGBoost Classifier...")
xgb_classifier = XGBClassifier(objective='multi:softmax', num_class=len(label_encoder.classes_), use_label_encoder=False, eval_metric='mlogloss', random_state=42)

In [ ]:
#Definindo Pipeline com oversampling e classifier XGB
pipeline_xgb = Pipeline([
    ('oversample', RandomOverSampler(random_state=42)),
    ('classifier', XGBClassifier(objective='multi:softmax', num_class=len(label_encoder.classes_), use_label_encoder=False, eval_metric='mlogloss', random_state=42))
])

#Ajustando o grid de hiperparâmetros
param_grid_xgb = {
    'classifier__n_estimators': [200], # Número de árvores
    'classifier__learning_rate': [0.1, 0.2], # Taxa de aprendizado
    'classifier__max_depth': [5, 7],      # Profundidade máxima das árvores
    'classifier__subsample': [0.9, 1.0],  # Fração de amostras para ajustar árvores individuais
    'classifier__colsample_bytree': [0.9, 1.0] # Fração de features para ajustar árvores individuais
}

# Configurando o GridSearchCV com validação cruzada
grid_search_xgb = GridSearchCV(
    pipeline_xgb,
    param_grid_xgb,
    cv=3,  # 5-fold cross-validation
    scoring='accuracy',  # Métrica de avaliação
    n_jobs=-1,  # Usar todos os cores disponíveis
    verbose=2  # Exibir progresso
)

print("\n Realizando ajuste de hiperparâmetros para XGBoost com validação cruzada...")

# Executando o GridSearchCV nos dados de treino escalados e codificados
grid_search_xgb.fit(X_train_scaled, y_train_encoded)

print("\n Ajuste de hiperparâmetros para XGBoost concluído!")

# Melhores parâmetros
print(grid_search_xgb.best_params_)

# Melhor acurácia
print(f"{grid_search_xgb.best_score_:.4f}")


🤖 Realizando ajuste de hiperparâmetros para XGBoost com validação cruzada...
Fitting 3 folds for each of 16 candidates, totalling 48 fits


In [ ]:
#Treino do modelo XGB
xgb_classifier.fit(X_train_scaled, y_train_encoded, verbose=True)

print("XGBoost Classifier training complete.")

In [ ]:
#Previsões
y_pred_xgb = xgb_classifier.predict(X_test_scaled)

In [ ]:
#Plotando matriz de confusão XGB
matriz_confusao = confusion_matrix(y_test_encoded, y_pred_xgb)
plt.figure(figsize=(20, 18))
sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Reds', cbar=True, annot_kws={"size": 14})
plt.xlabel('Valores previstos XGB', fontsize=20)
plt.ylabel('Valores reais XGB', fontsize=20)
plt.show()

In [ ]:
#Relatório
report_xgb = classification_report(y_test_encoded, y_pred_xgb, output_dict=True)
df_report_xgb = pd.DataFrame(report_xgb).transpose()
print("\nClassification Report (XGBoost) as DataFrame:")
df_report_xgb

In [ ]:
#Calculando RMSE
rmse_xgb = np.sqrt(mean_squared_error(y_test_encoded, y_pred_xgb))
print(f"\nRMSE (XGBoost): {rmse_xgb:.4f}")

In [ ]:
#calculando MAE
mae_xgb = mean_absolute_error(y_test_encoded, y_pred_xgb)
print(f"\nMAE (XGBoost): {mae_xgb:.4f}")

# **Comparação de desempenho**

In [ ]:
#Calculando métricas para RF
accuracy_rf = accuracy_score(y_test_encoded, y_pred)
f1_rf = f1_score(y_test_encoded, y_pred, average='weighted')

# Calculando métricas para SVM
accuracy_svm = accuracy_score(y_test_encoded, y_pred_svm)
f1_svm = f1_score(y_test_encoded, y_pred_svm, average='weighted')

# Calculando métricas para XGBoost
accuracy_xgb = accuracy_score(y_test_encoded, y_pred_xgb)
f1_xgb = f1_score(y_test_encoded, y_pred_xgb, average='weighted')

# Criando um DataFrame para armazenar as métricas
metrics_data = {
    'Algorithm': ['Random Forest', 'SVM', 'XGBoost'],
    'Accuracy': [accuracy_rf, accuracy_svm, accuracy_xgb],
    'F1-Score': [f1_rf, f1_svm, f1_xgb],
    'RMSE': [rmse_rfc, rmse_svm, rmse_xgb],
    'MAE': [mae_rfc, mae_svm, mae_xgb]
}
metrics_df = pd.DataFrame(metrics_data)

print(metrics_df)

# Plotando o gráfico comparativo
metrics_df_melted = metrics_df.melt(id_vars='Algorithm', var_name='Metric', value_name='Score')

cores_algoritmos = {
    'Random Forest': 'green',
    'SVM': 'blue',
    'XGBoost': 'red'
}

plt.figure(figsize=(16, 9))
sns.barplot(x='Metric', y='Score', hue='Algorithm', data=metrics_df_melted, palette=cores_algoritmos)

plt.title('Comparação de desempenho', fontsize=20)
plt.xlabel('Métrica', fontsize=15)
plt.xticks(fontsize=15)
plt.ylabel('Score', fontsize=15)
plt.yticks(fontsize=15)
plt.legend(title='Algoritmo')
plt.grid(axis='y', linestyle='--')
plt.show()

# **Redução de dimensionalidade (SVM)**

In [ ]:
# Aplicar LDA para redução de dimensionalidade
n_components_lda = min(len(label_encoder.classes_) - 1, X_train_scaled.shape[1])
if n_components_lda <= 0:
  print("Cannot apply LDA: Number of components must be greater than 0.")
else:
  lda = LinearDiscriminantAnalysis(n_components=n_components_lda)

  X_train_lda = lda.fit_transform(X_train_scaled, y_train_encoded)
  X_test_lda = lda.transform(X_test_scaled)

  print(f"\n Dimensionalidade reduzida com LDA para {n_components_lda} componentes.")
  print("X_train_lda shape:", X_train_lda.shape)
  print("X_test_lda shape:", X_test_lda.shape)

  # Treinar SVM nos dados com dimensionalidade reduzida
  print("\n Treinando SVM nos dados com dimensionalidade reduzida (LDA)...")
  svm_classifier_lda = SVC(random_state=42)
  svm_classifier_lda.fit(X_train_lda, y_train_encoded)
  print("SVM Classifier with LDA training complete.")

  # Previsões
  y_pred_svm_lda = svm_classifier_lda.predict(X_test_lda)

  # Avaliação
  print("\n Avaliação do SVM com LDA (sem ajuste de hiperparâmetros):")
  accuracy_svm_lda = accuracy_score(y_test_encoded, y_pred_svm_lda)
  f1_svm_lda = f1_score(y_test_encoded, y_pred_svm_lda, average='weighted')
  rmse_svm_lda = np.sqrt(mean_squared_error(y_test_encoded, y_pred_svm_lda))
  mae_svm_lda = mean_absolute_error(y_test_encoded, y_pred_svm_lda)

  print(f"Accuracy: {accuracy_svm_lda:.4f}")
  print(f"F1-Score (Weighted): {f1_svm_lda:.4f}")
  print(f"RMSE: {rmse_svm_lda:.4f}")
  print(f"MAE: {mae_svm_lda:.4f}")

  # Matriz de Confusão SVM LDA
  matriz_confusao_lda = confusion_matrix(y_test_encoded, y_pred_svm_lda)
  plt.figure(figsize=(20, 18))
  sns.heatmap(matriz_confusao_lda, annot=True, fmt='d', cmap='Blues', cbar=True, annot_kws={"size": 16})
  plt.xlabel('Valores previstos SVM (LDA)', fontsize=25)
  plt.ylabel('Valores reais SVM (LDA)', fontsize=25)
  plt.title('Matriz de Confusão - SVM com LDA', fontsize=25)
  plt.show()

  # Recalculando métricas
  accuracy_svm = accuracy_score(y_test_encoded, svm_classifier.predict(X_test_scaled))
  f1_svm = f1_score(y_test_encoded, svm_classifier.predict(X_test_scaled), average='weighted')
  rmse_svm = np.sqrt(mean_squared_error(y_test_encoded, svm_classifier.predict(X_test_scaled)))
  mae_svm = mean_absolute_error(y_test_encoded, svm_classifier.predict(X_test_scaled))

  # Comparando com original
  print("\nComparação com o SVM original (sem LDA):")
  print(f"Original SVM Accuracy: {accuracy_svm:.4f}")
  print(f"SVM with LDA Accuracy: {accuracy_svm_lda:.4f}")
  print(f"Original SVM F1-Score: {f1_svm:.4f}")
  print(f"SVM with LDA F1-Score: {f1_svm_lda:.4f}")
  print(f"Original SVM RMSE: {rmse_svm:.4f}")
  print(f"SVM with LDA RMSE: {rmse_svm_lda:.4f}")
  print(f"Original SVM MAE: {mae_svm:.4f}")
  print(f"SVM with LDA MAE: {mae_svm_lda:.4f}")